# Smartphone's Screen Crack Detection and Scoring using EfficientNet

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators with augmentation for training set
train_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  validation_split=0.2,
  rotation_range=15,
  width_shift_range=0.1,
  height_shift_range=0.1,
  brightness_range=[0.8, 1.2],
  zoom_range=0.1,
  horizontal_flip=True
)

# Define data generator for validation set
val_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  validation_split=0.2
)

# Define data generators
train_gen = train_datagen.flow_from_directory(
  "data",
  target_size=(224,224),
  batch_size=16,
  class_mode="categorical",
  subset="training",
  shuffle=True
)

# Define validation data generator
val_gen = val_datagen.flow_from_directory(
  "data",
  target_size=(224,224),
  batch_size=16,
  class_mode="categorical",
  subset="validation",
  shuffle=False
)


Found 236 images belonging to 5 classes.
Found 56 images belonging to 5 classes.


In [22]:
# Check training label
print(train_gen.class_indices)

{'broken': 0, 'damaged': 1, 'safe': 2, 'severe': 3, 'warning': 4}


In [ ]:
from pathlib import Path
from PIL import Image

root = Path("data")

bad_files = []

for img_path in root.rglob("*.jpg"):
  try:
    with Image.open(img_path) as img:
      img.verify()  # cek integritas file
  except Exception as e:
      print("BAD:", img_path, "->", e)
      bad_files.append(img_path)

print("\nTotal bad files:", len(bad_files))



Total bad files: 0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define model
base = tf.keras.applications.EfficientNetB0(
  include_top=False, weights="imagenet",
  input_shape=(224, 224, 3), pooling="avg"
)

# Backbone freeze
for layer in base.layers:
  layer.trainable = False

# Define classification head
model = layers.Dense(256, activation="relu")(base.output)
model = layers.Dropout(0.4)(model)
output = layers.Dense(5, activation="softmax")(model)

# Create final model
model = models.Model(inputs=base.input, outputs=output)

# Compile and train   model
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss="categorical_crossentropy",
  metrics=["accuracy"]
)

# Fit model using 10 epochs with early stopping
history = model.fit(
  train_gen,
  epochs=10,
  validation_data=val_gen,
  callbacks=[
    tf.keras.callbacks.EarlyStopping(
      patience=3, restore_best_weights=True, monitor="val_loss"
    )
  ]
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.2669 - loss: 1.6210 - val_accuracy: 0.2679 - val_loss: 1.6078
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4492 - loss: 1.2380 - val_accuracy: 0.3571 - val_loss: 1.5473
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 999ms/step - accuracy: 0.6059 - loss: 1.0010 - val_accuracy: 0.3036 - val_loss: 1.5533
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.7500 - loss: 0.7870 - val_accuracy: 0.3214 - val_loss: 1.6236
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.7797 - loss: 0.7053 - val_accuracy: 0.4107 - val_loss: 1.7102


In [28]:
# Fine tune model for last 40 layers
for layer in base.layers[-40:]:   
  layer.trainable = True

# Callbacks for saving best model and early stopping
checkpoint = tf.keras.callbacks.ModelCheckpoint(
  "exports/model_effnet.h5", monitor="val_loss",
  save_best_only=True, mode="max"
)

# Early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
  patience=3, restore_best_weights=True, monitor="val_loss"
)

# Compile model using a lower learning rate
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-4),
  loss="categorical_crossentropy",
  metrics=["accuracy"]
)

# Continue training the model
history2 = model.fit(
  train_gen,
  epochs=20,
  validation_data=val_gen,
  callbacks=[checkpoint, early_stop]
)


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7442 - loss: 0.9291

15/15 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.7331 - loss: 0.9240 - val_accuracy: 0.3929 - val_loss: 1.5259
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7437 - loss: 0.8633

15/15 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.7373 - loss: 0.8656 - val_accuracy: 0.3929 - val_loss: 1.5270
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7864 - loss: 0.7947

15/15 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.7797 - loss: 0.7760 - val_accuracy: 0.3750 - val_loss: 1.5340
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8472 - loss: 0.6508

15/15 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.8432 - loss: 0.6669 - val_accuracy: 0.3750 - val_loss: 1.5557
